In [223]:
from my_weapon import *
from SQLite_handler import *
from tqdm.notebook import tqdm as tqdm

plt.style.use('fivethirtyeight')

# Rural and Urban

In [231]:
rural_table = pd.read_csv("data/ruralurbancodes2013.csv")
rural_table["Population"] = rural_table["Population_2010"].apply(lambda x: int(x.replace(",", "")))
rural_table["is_rural"] = rural_table["RUCC_2013"].apply(lambda x: 1 if x >= 5 else 0)
# rural_table[rural_table.County_Name=="Orleans Parish"]

# def county_to_rural(state_name, county_name):
#     try:
#         return rural_table[(rural_table["State"]==state_name) & (rural_table["county"]==county_name)]["is_rural"].iloc[0]
#     except:
#         return f"cannot find the loc ({state_name}, {county_name})"

from collections import defaultdict
dict_state_county_is_rural = defaultdict(dict)

for row in rural_table.itertuples():
    State = getattr(row, "State")
    county = getattr(row, "County_Name")
    is_rural = getattr(row, "is_rural")
    if county != "District of Columbia":
        county = " ".join(county.split()[: -1])
    dict_state_county_is_rural[State][county] = is_rural
    
# print(dict_state_county_is_rural["LA"])

rural_population = rural_table.groupby(["State", "is_rural"]).sum()["Population"]
rural_count = dict(rural_population)

In [225]:
rural_table

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description,Population,is_rural
0,1001,AL,Autauga County,"54,571",2,"Metro - Counties in metro areas of 250,000 to ...",54571,0
1,1003,AL,Baldwin County,"182,265",3,Metro - Counties in metro areas of fewer than ...,182265,1
2,1005,AL,Barbour County,"27,457",6,"Nonmetro - Urban population of 2,500 to 19,999...",27457,1
3,1007,AL,Bibb County,"22,915",1,Metro - Counties in metro areas of 1 million p...,22915,0
4,1009,AL,Blount County,"57,322",1,Metro - Counties in metro areas of 1 million p...,57322,0
...,...,...,...,...,...,...,...,...
3216,72145,PR,Vega Baja Municipio,"59,662",1,Metro - Counties in metro areas of 1 million p...,59662,0
3217,72147,PR,Vieques Municipio,"9,301",7,"Nonmetro - Urban population of 2,500 to 19,999...",9301,1
3218,72149,PR,Villalba Municipio,"26,073",2,"Metro - Counties in metro areas of 250,000 to ...",26073,0
3219,72151,PR,Yabucoa Municipio,"37,941",1,Metro - Counties in metro areas of 1 million p...,37941,0


## Load the users' opinions and locations

In [232]:
users_loc = pd.read_csv("data/us2016-users-location.csv", usecols=["uid", "state", "county"], index_col="uid")
users_loc = users_loc.dropna()
users_loc["county"] = users_loc["county"].apply(lambda x: " ".join(x.split(", ")[:-1]))
users_loc

,state,county
uid,,
859057183,FL,Jacksonville
236247352,NE,Lincoln
57101844,CA,Walnut Creek
565112799,CA,Newport Beach
21247408,CA,Los Angeles
...,...,...
772611512544464896,GA,Marietta
4841885222,TX,Houston
22695927,MA,Boston


In [233]:
USA_STATES = ['CA', 'TX', 'NY', 'FL', 'IL', 'GA',
  'PA', 'OH', 'DC', 'NC', 'MI', 'MA',
  'IN', 'NJ', 'VA', 'AZ', 'TN', 'WA',
  'MD', 'CO', 'MO', 'KY', 'LA', 'MN',
  'OR', 'AL', 'SC', 'NV', 'OK', 'WI',
  'IA', 'CT', 'KS', 'AR', 'UT', 'MS',
  'WV', 'NE', 'NM', 'HI', 'NH', 'RI',
  'ME', 'ID', 'AK', 'DE', 'MT', 'SD',
  'ND', 'VT', 'WY']

In [234]:
# Since we only know the name of city, but the county. We need convert the city to county.
# check some mapping from http://www.statsamerica.org/CityCountyFinder/Default.aspx

_dict_city_to_county = {
    'Chicago, IL': 'Cook',
    'Washington, DC': 'District of Columbia',
    'Atlanta, GA': 'Fulton',
    'Boston, MA': 'Suffolk',
    'Seattle, WA': 'King',
    'Brooklyn, NY': 'New York',
    'Miami, FL': 'Miami-Dade',
    'Las Vegas, NV': 'Clark',
    'Portland, OR': 'Multnomah',
    'Pittsburgh, PA': 'Allegheny',
    'San Antonio, TX': 'Bexar',
    'Nashville, TN': 'Davidson',
    'Orlando, FL': 'Orange',
    'Phoenix, AZ': 'Maricopa',
    'Indianapolis, IN': 'Marion',
    'Columbus, OH': 'Franklin',
    'Minneapolis, MN': 'Hennepin',
    'St Louis, MO': 'St. Louis',
    'Charlotte, NC': 'Mecklenburg',
    'Cleveland, OH': 'Cuyahoga',
    'Detroit, MI': 'Wayne',
    'Tampa, FL': 'Hillsborough',
    'New Orleans, LA': 'Orleans',
    'Kansas City, MO': 'Jackson',
    'Cincinnati, OH': 'Hamilton'
}
    
def city_to_county(state_name, city_name):
    return _dict_city_to_county.get(f"{city_name}, {state_name}", city_name)

In [235]:
find_county = Counter()
cannot_find_county = Counter()

for _state in tqdm(USA_STATES):
    _users = users_loc[users_loc.state==_state].copy()
    _users["county"] = _users["county"].apply(lambda x: city_to_county(_state, x))
    _users["is_rural"] = _users["county"].apply(lambda x: dict_state_county_is_rural[_state].get(x, -1))
    _users.to_csv(f"data/is_rural/{_state}.csv")

#     for row in _users.itertuples():
#         county = getattr(row, "county")
#         is_rural = getattr(row, "is_rural")
#         if is_rural == -1:
#             cannot_find_county[f"{county}, {_state}"] += 1
#         else:
#             find_county[f"{county}, {_state}"] += 1
    
    
# -1 represents that we cannot map the city (from BingMap) to county
# So we need more infomation between city and county

In [ ]:
len(cannot_find_county)

In [97]:
cannot_find_county.most_common(10)

[('Louisville, KY', 8045),
 ('Hell, MI', 8033),
 ('Raleigh, NC', 8005),
 ('Buffalo, NY', 7353),
 ('Fort Worth, TX', 7115),
 ('Memphis, TN', 6850),
 ('Oakland, CA', 6527),
 ('Salt Lake City, UT', 5938),
 ('Manhattan, NY', 5921),
 ('San Jose, CA', 5732)]

In [98]:
find_county.most_common(10)

[('New York, NY', 148428),
 ('Los Angeles, CA', 92810),
 ('Cook, IL', 62030),
 ('District of Columbia, DC', 49085),
 ('Houston, TX', 44915),
 ('Fulton, GA', 36245),
 ('Dallas, TX', 30003),
 ('San Francisco, CA', 28725),
 ('Suffolk, MA', 27881),
 ('Philadelphia, PA', 27672)]

In [204]:
# load users' opinion
# users_opinion = pd.read_csv("data/tweetid_userid_pro_hillary.csv")
import pendulum
users_opinion = defaultdict(lambda: [0, 0])
set_users = set(users_loc.index)
# start = pendulum.datetime(2016, 9, 1)

# election date: 2016-11-8
for line in tqdm(open("data/tweetid_userid_pro_hillary.csv")):
    w = line.strip().split(",")
    uid = int(w[0])
    if uid in set_users:
        if float(w[2]) < 0.5:
            users_opinion[uid][0] += 1
        else:
            users_opinion[uid][1] += 1

In [205]:
users_opinion_v2 = []
for u, v in tqdm(users_opinion.items()):
    users_opinion_v2.append({
        "uid": u,
        "Camp": "T" if v[0] > v[1] else "H" 
    })

In [206]:
users_opinion = pd.DataFrame(users_opinion_v2).set_index("uid")
users_opinion

,Camp
uid,
18808729,H
312876456,H
11140312,H
15363308,H
10236782,H
...,...
4841885222,H
130932192,H
22695927,T


In [237]:
for _state in tqdm(USA_STATES):
    users_county = pd.read_csv("data/is_rural/" + _state + ".csv", index_col="uid")
    users_county = users_county.join(users_opinion)
    groups = dict(users_county.groupby(["is_rural", "Camp"]).size())    

    w1 = rural_count.get((_state, 1), 0) / (rural_count.get((_state, 1), 0) + rural_count.get((_state, 0), 0))
    w0 = 1 - w1
    
    groups_T_H = dict(users_county[users_county.is_rural != -1].groupby("Camp").size())
    # print(groups_T_H)
    proT = groups_T_H.get("T", 0) / (groups_T_H.get("T", 0) + groups_T_H.get("H", 0))
    

    if (0, 'T') not in groups: # No urban people support Trump!
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'H')])
        rst = pro_T_1
    elif (1, 'T') not in groups: # No rural people support Trump!
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'H')])
        rst = pro_T_0
    else:
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'H')])
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'H')])
        rst = pro_T_1 * w1 + pro_T_0 * w0
        
    print(f"{_state}, Census (rural areas): {w1 * 100:.2f}%, Trump: {proT * 100:.2f}%, After rescaling: {rst * 100:.2f}%")

CA, Census (rural areas): 1.28%, Trump: 17.79%, After rescaling: 18.03%
TX, Census (rural areas): 9.44%, Trump: 24.11%, After rescaling: 24.26%
NY, Census (rural areas): 3.21%, Trump: 18.61%, After rescaling: 18.90%
FL, Census (rural areas): 2.20%, Trump: 26.35%, After rescaling: 26.78%
IL, Census (rural areas): 8.17%, Trump: 18.80%, After rescaling: 20.81%
GA, Census (rural areas): 13.82%, Trump: 21.76%, After rescaling: 22.59%
PA, Census (rural areas): 5.14%, Trump: 23.33%, After rescaling: 24.00%
OH, Census (rural areas): 7.61%, Trump: 25.42%, After rescaling: 26.89%
DC, Census (rural areas): 0.00%, Trump: 16.86%, After rescaling: 16.86%
NC, Census (rural areas): 11.80%, Trump: 24.25%, After rescaling: 26.09%
MI, Census (rural areas): 13.54%, Trump: 23.97%, After rescaling: 24.95%
MA, Census (rural areas): 0.41%, Trump: 18.94%, After rescaling: 18.96%
IN, Census (rural areas): 17.87%, Trump: 26.44%, After rescaling: 28.78%
NJ, Census (rural areas): 0.00%, Trump: 29.17%, After rescal